In [1]:
import random
import time
from collections import OrderedDict

from tabulate import tabulate

In [2]:
%run brute_force.ipynb
%run ga.ipynb
%run vns.ipynb

In [3]:
class TestCase:
    def __init__(self, category, name, alphabet, words, verify_with_brute_force):
        self.category = category
        self.name = name
        self.alphabet = alphabet
        self.words = words
        self.verify_with_bf = verify_with_brute_force

    @property
    def n(self):
        return len(self.words)

    @property
    def m(self):
        return len(self.words[0]) if self.words else 0

    def run(self):
        results = OrderedDict()

        # GA
        start = time.perf_counter()
        ga_min_distance, _ = ga(self.words)
        results["GA"] = {
            "distance": ga_min_distance,
            "time": time.perf_counter() - start,
        }

        # VNS
        start = time.perf_counter()
        vns_min_distance, _ = vns(self.words)
        results["VNS"] = {
            "distance": vns_min_distance,
            "time": time.perf_counter() - start,
        }

        # BF (opciono)
        if self.verify_with_bf:
            start = time.perf_counter()
            bf_min_distance, _ = brute_force(self.words, self.alphabet)
            results["Brute Force"] = {
                "distance": bf_min_distance,
                "time": time.perf_counter() - start,
            }
        else:
            results["Brute Force"] = {"distance": "N/A", "time": "N/A"}

        return results

In [4]:
def _generate_string(m, alphabet):
    return "".join(random.choice(alphabet) for _ in range(m))

In [5]:
def _generate_instance(n, m, alphabet):
    return [_generate_string(m, alphabet) for _ in range(n)]

In [6]:
def _generate_tests():
    tests = []

    tests.append(
        TestCase("Trivijalni", "Identični stringovi", "ABC", ["AAAAA"] * 10, True)
    )

    tests.append(TestCase("Trivijalni", "Jedan string", "AB", ["ABABA"], True))

    tests.append(
        TestCase("Granični slučaj", "Binarni komplement", "AB", ["AAAA", "BBBB"], True)
    )

    tests.append(
        TestCase(
            "Granični slučaj",
            "Kolonska većina",
            "ABC",
            ["ABCABC", "ABCABC", "ABCACC", "ABCABC", "ABCABC"],
            True,
        )
    )

    tests.append(
        TestCase(
            "Granični slučaj",
            "Dve suprotne grupe",
            "AB",
            ["AAAAAA"] * 50 + ["BBBBBB"] * 50,
            True,
        )
    )

    # Mali slučajni testovi (brute-force izvodljiv)
    for _ in range(3):
        n = random.randint(5, 8)
        m = random.randint(5, 7)
        alphabet = "ABC"
        words = _generate_instance(n, m, alphabet)
        tests.append(
            TestCase(
                "Mali br. instanci",
                f"Validacija sa BF",
                alphabet,
                words,
                True,
            )
        )

    # Srednji test – BF ~ desetine minuta
    tests.append(
        TestCase(
            "Srednji br. instanci",
            "Performanse u odnosu na BF",
            "ABCDEFGH",
            _generate_instance(25, 8, "ABCDEFGH"),
            True,
        )
    )

    # Veliki test – samo metaheuristike
    tests.append(
        TestCase(
            "Veliki br. instanci",
            "Test performansi",
            "QWERTYUIOPASDFGHJKLZXCVBNM",
            _generate_instance(1000, 12, "QWERTYUIOPASDFGHJKLZXCVBNM"),
            False,
        )
    )

    # Uticaj veličine azbuke
    alphabets = [
        "AB",
        "ABC",
        "ABCDE",
        "ABCDEFGHIJ",
        "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    ]
    for sigma in alphabets:
        tests.append(
            TestCase(
                "Uticaj |Σ|",
                f"|Σ| = {len(sigma)}",
                sigma,
                _generate_instance(100, 12, sigma),
                False,
            )
        )

    # Uticaj broja instanci
    for n in [10, 50, 100, 500, 1000]:
        tests.append(
            TestCase(
                "Uticaj n",
                f"n = {n}",
                alphabet,
                _generate_instance(n, 12, "ABCD"),
                False,
            )
        )

    # Uticaj dužine stringa
    for m in [1, 5, 10, 20, 50]:
        tests.append(
            TestCase(
                "Uticaj m",
                f"m = {m}",
                alphabet,
                _generate_instance(100, m, "ABCD"),
                False,
            )
        )

    return tests

In [7]:
def run_tests():
    tests = _generate_tests()
    table = []

    for test in tests:
        results = test.run()

        row = [
            test.category,
            test.name,
            f"{len(test.alphabet)}",
            f"{test.n}",
            f"{test.m}",
            results["Brute Force"]["distance"],
            (
                f'{results["Brute Force"]["time"]:.3f}s'
                if results["Brute Force"]["time"] != "N/A"
                else "N/A"
            ),
            results["GA"]["distance"],
            f'{results["GA"]["time"]:.3f}s',
            results["VNS"]["distance"],
            f'{results["VNS"]["time"]:.3f}s',
        ]

        table.append(row)

    headers = [
        "Kategorija testa",
        "Opis testa",
        "|Σ|",
        "n",
        "m",
        "BF dist",
        "BF vreme",
        "GA dist",
        "GA vreme",
        "VNS dist",
        "VNS vreme",
    ]

    print("\n\n\n")
    print(
        tabulate(
            table,
            headers=headers,
            tablefmt="github",
            colalign=("center",) * len(headers),
        )
    )